## CNN для расопзнавания символов

In [1]:
from keras import utils
from keras.datasets import mnist # dataset
from keras.models import Sequential
from keras.layers import Dense, Dropout, Conv2D, MaxPooling2D, Reshape
from keras.optimizers import Adam
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from keras import regularizers
from keras.models import load_model

Using TensorFlow backend.


In [2]:
batch_size = 8
num_classes = 62
epochs = 10

In [3]:
def batch_loader(path, id_from=0, id_to=100):
    for i in range(id_from, id_to):
        X = np.load(path + '/images/{}.npy'.format(i))
        X.dtype=np.uint8
        y = np.load(path + '/answers/{}.npy'.format(i))
        yield np.unpackbits(X, axis = 2), y
        
def generator_for_CNN(path, LB, id_from=0, id_to=100):
    i = id_from
    while 1:
        print('GENRUN', i)
        if i == id_to:
            i = id_from
        X = np.load(path + '/images/{}.npy'.format(i))
        X.dtype=np.uint8
        y = np.load(path + '/answers/{}.npy'.format(i))
        i += 1
        X = np.unpackbits(X, axis = 2)
        yield X.reshape(*X.shape, 1), LB.transform(y)


In [4]:
LB = LabelBinarizer()
for X, y in batch_loader('Generator/generated', 0, 1):
    X_train, y_train = X.reshape(*X.shape, 1), LB.fit_transform(y)

for X, y in batch_loader('Generator/generated', 1, 2):
    X_test, y_test = X.reshape(*X.shape, 1), LB.transform(y)

In [5]:

# create model
model = Sequential()

model.add(Conv2D(filters = batch_size, 
                         kernel_size=3, 
                         activation='relu', 
                         padding='same',
                         input_shape=(64, 64, 1),
                         data_format='channels_last'))
for i in range(2):
    for j in range(2):
        model.add(Conv2D(filters = batch_size, 
                         kernel_size=3, 
                         padding='same',
                         activation='relu'))
        model.add(Dropout(0.2))
    model.add(MaxPooling2D())
model.add(Reshape([256 * batch_size]))
model.add(Dense(num_classes, 
                activation='softmax'
               )
         )

# print model summarization
model.summary()

model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 64, 64, 8)         80        
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 64, 64, 8)         584       
_________________________________________________________________
dropout_1 (Dropout)          (None, 64, 64, 8)         0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 64, 64, 8)         584       
_________________________________________________________________
dropout_2 (Dropout)          (None, 64, 64, 8)         0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 32, 32, 8)         0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 32, 32, 8)         584       
__________

In [6]:
history = model.fit(X_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    validation_data=(X_test, y_test))
# evaluate model
score = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Train on 10000 samples, validate on 10000 samples
Epoch 1/10
10000/10000 [==============================] - 63s 6ms/step - loss: 1.7528 - acc: 0.5438 - val_loss: 1.1191 - val_acc: 0.7616
Epoch 2/10
10000/10000 [==============================] - 60s 6ms/step - loss: 0.6921 - acc: 0.8014 - val_loss: 0.8689 - val_acc: 0.7781
Epoch 3/10
10000/10000 [==============================] - 62s 6ms/step - loss: 0.5039 - acc: 0.8485 - val_loss: 0.7023 - val_acc: 0.8112
Epoch 4/10
10000/10000 [==============================] - 63s 6ms/step - loss: 0.3932 - acc: 0.8740 - val_loss: 0.7029 - val_acc: 0.8253
Epoch 5/10
10000/10000 [==============================] - 63s 6ms/step - loss: 0.3161 - acc: 0.8991 - val_loss: 0.6133 - val_acc: 0.8356
Epoch 6/10
10000/10000 [==============================] - 65s 6ms/step - loss: 0.2612 - acc: 0.9142 - val_loss: 0.5650 - val_acc: 0.8420
Epoch 7/10
10000/10000 [==============================] - 64s 6ms/step - loss: 0.2343 - acc: 0.9198 - val_loss: 0.5567 - val_acc

In [8]:
model.save('base_model.h5')